## Use PyQuery

### - Use selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import csv 
from pyquery import PyQuery as pq
from multiprocessing import Pool
import os

driver = webdriver.Chrome('D:\\Github\\MovieCrawler\\chromedriver.exe')
url_list = "https://www.asus.com/us/Motherboards/AllProducts/"

driver.get(url_list)
html = driver.page_source
doc = pq(html)
driver.close()


### result : too slow

### - Use Header

In [2]:
#User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36
doc3 = pq(
    'https://www.asus.com/us/Motherboards/AllProducts/', 
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36'})
print(type(doc3))

<class 'pyquery.pyquery.PyQuery'>


### fail 依舊 403 forbidden

### - Use requests


In [3]:
import requests

response = requests.get('https://www.asus.com/us/Motherboards/AllProducts/')
doc2 = pq(response.text)


### 用Dictionary來存網址, 避免重複

In [4]:
Mb_URL_dic = {}
ul_list = doc2('.unstyled')
li_list = ul_list('li')
for li in li_list.items():
    herf = li.find('a').attr('href')
    title = li.find('a').text()
    Mb_URL_dic[title] = ('https://www.asus.com' + herf + 'specifications/')
Mb_URL_dic


{'B150M-A/M.2': 'https://www.asus.com/us/Motherboards/B150M-A-M-2/specifications/',
 'B150M-PLUS D3': 'https://www.asus.com/us/Motherboards/B150M-PLUS-D3/specifications/',
 'B250 MINING EXPERT': 'https://www.asus.com/us/Motherboards/B250-MINING-EXPERT/specifications/',
 'EX-B250-V7': 'https://www.asus.com/us/Motherboards/EX-B250-V7/specifications/',
 'H110I-PLUS/CSM': 'https://www.asus.com/us/Motherboards/H110I-PLUS-CSM/specifications/',
 'H110M-A/DP': 'https://www.asus.com/us/Motherboards/H110M-A-DP/specifications/',
 'H110M-A/M.2': 'https://www.asus.com/us/Motherboards/H110M-A-M-2/specifications/',
 'H110M-C/CSM': 'https://www.asus.com/us/Motherboards/H110M-C-CSM/specifications/',
 'H110M-E/M.2': 'https://www.asus.com/us/Motherboards/H110M-E-M-2/specifications/',
 'H110M-K': 'https://www.asus.com/us/Motherboards/H110M-K/specifications/',
 'H81I-PLUS/CSM': 'https://www.asus.com/us/Motherboards/H81I-PLUS-CSM/specifications/',
 'H81M-A': 'https://www.asus.com/us/Motherboards/H81MA/speci

### 抓SPEC

In [5]:
%%time
df = pd.DataFrame()
url_link = list(Mb_URL_dic.values())[0]
print(url_link)
response = requests.get(url_link)
doc2 = pq(response.text)
title_list = doc2('.spec-item')


# for link in Mb_URL_dic.values():
#     response = requests.get(link)
#     doc2 = pq(response.text)
#     title_list = doc2('spec-item')
#     print(type(title_list))

https://www.asus.com/us/Motherboards/Z97AUSB_31/specifications/
Wall time: 99 ms


In [ ]:
%%time


## create DataFrame with title list
df = pd.DataFrame()

for link in Mb_URL_dic.values():
    response = requests.get(link)
    doc2 = pq(response.text)
    
    pq_title_list = doc2('.spec-item')
    pq_data_list = doc2('.spec-data')
    MB_name = doc2('.page-title').text()

    title_list =['Board name']
    data_list = [MB_name]
    print("test : " + MB_name)

    for title in pq_title_list:
        title_list.append(title.text)
    for data in pq_data_list:
        data_list.append(data.text)
    
#     print(title_list)
#     print(data_list)
    
    try :
        df = df.append(pd.Series(data_list,index=title_list),ignore_index=True)
    except ValueError:
        print("Error MB page : " + MB_name)
df



## 再度嘗試headers

In [8]:
%%time

## create DataFrame with title list
df_header = pd.DataFrame()

for link in Mb_URL_dic.values():
    doc_header = pq(
        link,
        headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36'})
    #print("test : " + doc_header.html())
    pq_title_list = doc_header('.spec-item')
    pq_data_list = doc_header('.spec-data')
    MB_name = doc_header('.page-title').text()
    
    title_list =['Board name']
    data_list = [MB_name]
    print("test : " + MB_name)

    for title in pq_title_list:
        title_list.append(title.text)
    for data in pq_data_list:
        data_list.append(data.text)
    
    
    try :
        df_header = df_header.append(pd.Series(data_list,index=title_list),ignore_index=True)
    except ValueError:
        print("Error MB page : " + MB_name)
df_header.head()

test : Z97-A/USB 3.1
test : PRIME X470-PRO
test : PRIME H310M-A
test : PRIME H310M-D
test : PRIME B360M-A
test : PRIME H310I-PLUS/CSM
test : PRIME H310M-C/CSM
test : PRIME B360M-C/CSM
test : PRIME H310M-K
test : PRIME H310M-E
test : PRIME B360M-K
test : PRIME H370M-PLUS/CSM
test : PRIME H370-PLUS/CSM
test : PRIME H370-A/CSM
test : PRIME B360-PLUS
test : PRIME H370M-PLUS
test : PRIME H370-PLUS
test : Q170M-CM-B/CSM Pro
test : H110I-PLUS/CSM
test : B150M-PLUS D3
test : H110M-C/CSM
test : Q170M-C/CSM
test : PRIME Q270M-C/CSM
test : PRIME J3355I-C
test : PRIME Z370-A
test : PRIME Z370-P
test : EX-B250-V7
test : PRIME B350M-E
test : PRIME X399-A
test : PRIME A320M-A
test : PRIME A320M-C R2.0
test : PRIME X299-A
test : PRIME X299-DELUXE
test : PRIME H110M-P
test : PRIME X370-A
test : PRIME A320M-K
test : PRIME B350-PLUS
test : PRIME X370-PRO
test : PRIME H270-PLUS/CSM
test : PRIME B250M-C/CSM
test : PRIME B250M-K
test : PRIME B250M-A
test : PRIME B250M-PLUS
test : PRIME B250-PLUS
test : PRIM

### 用request + pyquery

In [15]:
%%timeit -n 5

# response = requests.get('https://www.asus.com/us/Motherboards/AllProducts/')
# doc2 = pq(response.text)

## create DataFrame with title list
df_req = pd.DataFrame()

for link in Mb_URL_dic.values():
    response = requests.get(link)
    doc_req = pq(response.text)
    #print("test : " + doc_header.html())
    pq_title_list = doc_req('.spec-item')
    pq_data_list = doc_req('.spec-data')
    MB_name = doc_req('.page-title').text()
    
    title_list =['Board name']
    data_list = [MB_name]
    print("test : " + MB_name)

    for title in pq_title_list:
        title_list.append(title.text)
    for data in pq_data_list:
        data_list.append(data.text)
    
    
    try :
        df_req = df_req.append(pd.Series(data_list,index=title_list),ignore_index=True)
    except ValueError:
        print("Error MB page : " + MB_name)
df_req.head()

test : Z97-A/USB 3.1
test : PRIME X470-PRO
test : PRIME H310M-A
test : PRIME H310M-D
test : PRIME B360M-A
test : PRIME H310I-PLUS/CSM
test : PRIME H310M-C/CSM
test : PRIME B360M-C/CSM
test : PRIME H310M-K
test : PRIME H310M-E
test : PRIME B360M-K
test : PRIME H370M-PLUS/CSM
test : PRIME H370-PLUS/CSM
test : PRIME H370-A/CSM
test : PRIME B360-PLUS
test : PRIME H370M-PLUS
test : PRIME H370-PLUS
test : Q170M-CM-B/CSM Pro
test : H110I-PLUS/CSM
test : B150M-PLUS D3
test : H110M-C/CSM
test : Q170M-C/CSM
test : PRIME Q270M-C/CSM
test : PRIME J3355I-C
test : PRIME Z370-A
test : PRIME Z370-P
test : EX-B250-V7
test : PRIME B350M-E
test : PRIME X399-A
test : PRIME A320M-A
test : PRIME A320M-C R2.0
test : PRIME X299-A
test : PRIME X299-DELUXE
test : PRIME H110M-P
test : PRIME X370-A
test : PRIME A320M-K
test : PRIME B350-PLUS
test : PRIME X370-PRO
test : PRIME H270-PLUS/CSM
test : PRIME B250M-C/CSM
test : PRIME B250M-K
test : PRIME B250M-A
test : PRIME B250M-PLUS
test : PRIME B250-PLUS
test : PRIM

test : PRIME H310M-E
test : PRIME B360M-K
test : PRIME H370M-PLUS/CSM
test : PRIME H370-PLUS/CSM
test : PRIME H370-A/CSM
test : PRIME B360-PLUS
test : PRIME H370M-PLUS
test : PRIME H370-PLUS
test : Q170M-CM-B/CSM Pro
test : H110I-PLUS/CSM
test : B150M-PLUS D3
test : H110M-C/CSM
test : Q170M-C/CSM
test : PRIME Q270M-C/CSM
test : PRIME J3355I-C
test : PRIME Z370-A
test : PRIME Z370-P
test : EX-B250-V7
test : PRIME B350M-E
test : PRIME X399-A
test : PRIME A320M-A
test : PRIME A320M-C R2.0
test : PRIME X299-A
test : PRIME X299-DELUXE
test : PRIME H110M-P
test : PRIME X370-A
test : PRIME A320M-K
test : PRIME B350-PLUS
test : PRIME X370-PRO
test : PRIME H270-PLUS/CSM
test : PRIME B250M-C/CSM
test : PRIME B250M-K
test : PRIME B250M-A
test : PRIME B250M-PLUS
test : PRIME B250-PLUS
test : PRIME H270-PRO
test : PRIME Z270M-PLUS
test : PRIME Z270-P
test : PRIME Z270-K
test : PRIME Z270-AR
test : PRIME Z270-A
test : H81T R2.0/CSM
test : H81I-PLUS/CSM
test : B150M-A/M.2
test : H110M-E/M.2
test : H1

test : H110I-PLUS/CSM
test : B150M-PLUS D3
test : H110M-C/CSM
test : Q170M-C/CSM
test : PRIME Q270M-C/CSM
test : PRIME J3355I-C
test : PRIME Z370-A
test : PRIME Z370-P
test : EX-B250-V7
test : PRIME B350M-E
test : PRIME X399-A
test : PRIME A320M-A
test : PRIME A320M-C R2.0
test : PRIME X299-A
test : PRIME X299-DELUXE
test : PRIME H110M-P
test : PRIME X370-A
test : PRIME A320M-K
test : PRIME B350-PLUS
test : PRIME X370-PRO
test : PRIME H270-PLUS/CSM
test : PRIME B250M-C/CSM
test : PRIME B250M-K
test : PRIME B250M-A
test : PRIME B250M-PLUS
test : PRIME B250-PLUS
test : PRIME H270-PRO
test : PRIME Z270M-PLUS
test : PRIME Z270-P
test : PRIME Z270-K
test : PRIME Z270-AR
test : PRIME Z270-A
test : H81T R2.0/CSM
test : H81I-PLUS/CSM
test : B150M-A/M.2
test : H110M-E/M.2
test : H110M-A/M.2
test : ROG STRIX X99 GAMING
test : X99-E
test : H110M-A/DP
test : H110M-K
test : Z170-E
test : Z170-PRO
test : Z170-K
test : X99-A/USB 3.1
test : X99-PRO/USB 3.1
test : Z97-E/USB3.1
test : H81M-E
test : H81M

test : PRIME X399-A
test : PRIME A320M-A
test : PRIME A320M-C R2.0
test : PRIME X299-A
test : PRIME X299-DELUXE
test : PRIME H110M-P
test : PRIME X370-A
test : PRIME A320M-K
test : PRIME B350-PLUS
test : PRIME X370-PRO
test : PRIME H270-PLUS/CSM
test : PRIME B250M-C/CSM
test : PRIME B250M-K
test : PRIME B250M-A
test : PRIME B250M-PLUS
test : PRIME B250-PLUS
test : PRIME H270-PRO
test : PRIME Z270M-PLUS
test : PRIME Z270-P
test : PRIME Z270-K
test : PRIME Z270-AR
test : PRIME Z270-A
test : H81T R2.0/CSM
test : H81I-PLUS/CSM
test : B150M-A/M.2
test : H110M-E/M.2
test : H110M-A/M.2
test : ROG STRIX X99 GAMING
test : X99-E
test : H110M-A/DP
test : H110M-K
test : Z170-E
test : Z170-PRO
test : Z170-K
test : X99-A/USB 3.1
test : X99-PRO/USB 3.1
test : Z97-E/USB3.1
test : H81M-E
test : H81M-A
test : M5A97 LE R2.0
test : M5A97 R2.0
test : ROG STRIX X370-F GAMING
test : PRIME H310-PLUS
test : ROG STRIX B360-H GAMING/OPTANE
test : ROG STRIX X470-F GAMING
test : ROG CROSSHAIR VII HERO
test : ROG C

test : PRIME X370-PRO
test : PRIME H270-PLUS/CSM
test : PRIME B250M-C/CSM
test : PRIME B250M-K
test : PRIME B250M-A
test : PRIME B250M-PLUS
test : PRIME B250-PLUS
test : PRIME H270-PRO
test : PRIME Z270M-PLUS
test : PRIME Z270-P
test : PRIME Z270-K
test : PRIME Z270-AR
test : PRIME Z270-A
test : H81T R2.0/CSM
test : H81I-PLUS/CSM
test : B150M-A/M.2
test : H110M-E/M.2
test : H110M-A/M.2
test : ROG STRIX X99 GAMING
test : X99-E
test : H110M-A/DP
test : H110M-K
test : Z170-E
test : Z170-PRO
test : Z170-K
test : X99-A/USB 3.1
test : X99-PRO/USB 3.1
test : Z97-E/USB3.1
test : H81M-E
test : H81M-A
test : M5A97 LE R2.0
test : M5A97 R2.0
test : ROG STRIX X370-F GAMING
test : PRIME H310-PLUS
test : ROG STRIX B360-H GAMING/OPTANE
test : ROG STRIX X470-F GAMING
test : ROG CROSSHAIR VII HERO
test : ROG CROSSHAIR VII HERO (WI-FI)
test : ROG STRIX B360-H GAMING
test : ROG STRIX B360-F GAMING
test : ROG STRIX H370-F GAMING
test : ROG STRIX B360-I GAMING
test : ROG STRIX B360-G GAMING
test : ROG STRIX

test : PRIME Z270-P
test : PRIME Z270-K
test : PRIME Z270-AR
test : PRIME Z270-A
test : H81T R2.0/CSM
test : H81I-PLUS/CSM
test : B150M-A/M.2
test : H110M-E/M.2
test : H110M-A/M.2
test : ROG STRIX X99 GAMING
test : X99-E
test : H110M-A/DP
test : H110M-K
test : Z170-E
test : Z170-PRO
test : Z170-K
test : X99-A/USB 3.1
test : X99-PRO/USB 3.1
test : Z97-E/USB3.1
test : H81M-E
test : H81M-A
test : M5A97 LE R2.0
test : M5A97 R2.0
test : ROG STRIX X370-F GAMING
test : PRIME H310-PLUS
test : ROG STRIX B360-H GAMING/OPTANE
test : ROG STRIX X470-F GAMING
test : ROG CROSSHAIR VII HERO
test : ROG CROSSHAIR VII HERO (WI-FI)
test : ROG STRIX B360-H GAMING
test : ROG STRIX B360-F GAMING
test : ROG STRIX H370-F GAMING
test : ROG STRIX B360-I GAMING
test : ROG STRIX B360-G GAMING
test : ROG STRIX H370-I GAMING
test : ROG STRIX B350-I GAMING
test : ROG STRIX X370-I GAMING
test : ROG STRIX Z370-G GAMING (WI-FI AC)
test : ROG MAXIMUS X APEX
test : ROG MAXIMUS X CODE
test : ROG MAXIMUS X FORMULA
test : RO

test : X99-E
test : H110M-A/DP
test : H110M-K
test : Z170-E
test : Z170-PRO
test : Z170-K
test : X99-A/USB 3.1
test : X99-PRO/USB 3.1
test : Z97-E/USB3.1
test : H81M-E
test : H81M-A
test : M5A97 LE R2.0
test : M5A97 R2.0
test : ROG STRIX X370-F GAMING
test : PRIME H310-PLUS
test : ROG STRIX B360-H GAMING/OPTANE
test : ROG STRIX X470-F GAMING
test : ROG CROSSHAIR VII HERO
test : ROG CROSSHAIR VII HERO (WI-FI)
test : ROG STRIX B360-H GAMING
test : ROG STRIX B360-F GAMING
test : ROG STRIX H370-F GAMING
test : ROG STRIX B360-I GAMING
test : ROG STRIX B360-G GAMING
test : ROG STRIX H370-I GAMING
test : ROG STRIX B350-I GAMING
test : ROG STRIX X370-I GAMING
test : ROG STRIX Z370-G GAMING (WI-FI AC)
test : ROG MAXIMUS X APEX
test : ROG MAXIMUS X CODE
test : ROG MAXIMUS X FORMULA
test : ROG MAXIMUS X HERO
test : ROG STRIX Z370-H GAMING
test : ROG STRIX Z370-F GAMING
test : ROG STRIX Z370-I GAMING
test : ROG STRIX Z370-E GAMING
test : ROG MAXIMUS X HERO (WI-FI AC)
test : ROG STRIX X299-XE GAMIN

test : M5A97 R2.0
test : ROG STRIX X370-F GAMING
test : PRIME H310-PLUS
test : ROG STRIX B360-H GAMING/OPTANE
test : ROG STRIX X470-F GAMING
test : ROG CROSSHAIR VII HERO
test : ROG CROSSHAIR VII HERO (WI-FI)
test : ROG STRIX B360-H GAMING
test : ROG STRIX B360-F GAMING
test : ROG STRIX H370-F GAMING
test : ROG STRIX B360-I GAMING
test : ROG STRIX B360-G GAMING
test : ROG STRIX H370-I GAMING
test : ROG STRIX B350-I GAMING
test : ROG STRIX X370-I GAMING
test : ROG STRIX Z370-G GAMING (WI-FI AC)
test : ROG MAXIMUS X APEX
test : ROG MAXIMUS X CODE
test : ROG MAXIMUS X FORMULA
test : ROG MAXIMUS X HERO
test : ROG STRIX Z370-H GAMING
test : ROG STRIX Z370-F GAMING
test : ROG STRIX Z370-I GAMING
test : ROG STRIX Z370-E GAMING
test : ROG MAXIMUS X HERO (WI-FI AC)
test : ROG STRIX X299-XE GAMING
test : ROG STRIX X399-E GAMING
test : ROG ZENITH EXTREME
test : ROG CROSSHAIR VI EXTREME
test : ROG RAMPAGE VI APEX
test : ROG RAMPAGE VI EXTREME
test : ROG STRIX X299-E GAMING
test : ROG STRIX B350-F 

In [11]:
df_req.head(10)

,Accessories,Audio,BIOS,Back I/O Ports,Board name,CPU,Chipset,Expansion Slots,Form Factor,Graphic,...,System Bus,ThunderBolt LAN/Audio Combo Card,ThunderFX,Thunderbolt,Workstation Feature,Intel® Optane™ Memroy Bundled,Environment,OC Panel II,Weight,Warranty
0,User's manual,Realtek® ALC892 8-Channel High Definition Audi...,"64 Mb Flash ROM, UEFI AMI BIOS, PnP, DMI2.7, W...",1 x PS/2 keyboard/mouse combo port(s),Z97-A/USB 3.1,Intel® Socket 1150 for the 5,Intel® Z97,2 x PCIe 3.0/2.0 x16 (x16 or dual x8),ATX Form Factor,Integrated Graphics Processor- Intel® HD Graph...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,User's manual,Realtek® S1220A 8-Channel High Definition Audi...,"256 Mb Flash ROM, UEFI AMI BIOS, PnP,SM BIOS 3...",1 x PS/2 keyboard/mouse combo port(s),PRIME X470-PRO,AMD AM4 Socket AMD Ryzen™ 2nd Generation/Ryze...,AMD X470,None,ATX Form Factor,Integrated Graphics in the AMD Ryzen™ with Rad...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,User's manual,Realtek® ALC887 8-Channel High Definition Audi...,"128 Mb Flash ROM, UEFI AMI BIOS, PnP, SM BIOS ...",1 x PS/2 keyboard (purple),PRIME H310M-A,Intel® Socket 1151 for 8,Intel® H310,"1 x PCIe 3.0/2.0 x16 (x16 mode, gray)",Micro ATX Form Factor,Integrated Graphics Processor- Intel® HD Graph...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,User's manual,Realtek® ALC887 8-Channel High Definition Audi...,"128 Mb Flash ROM, UEFI AMI BIOS, PnP, SM BIOS ...",1 x PS/2 keyboard/mouse combo port(s),PRIME H310M-D,Intel® Socket 1151 for 8,Intel® H310,"1 x PCIe 3.0/2.0 x16 (x16 mode, gray)",mATX Form Factor,Integrated Graphics Processor- Intel® HD Graph...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,User's manual,Realtek® ALC887 8-Channel High Definition Audi...,"128Mb Flash ROM, UEFI AMI BIOS, PnP, SM BIOS 3...",1 x PS/2 keyboard (purple),PRIME B360M-A,Intel® Socket 1151 for 8,Intel® B360,1 x PCIe 3.0 x16 (x16 mode),Micro ATX Form Factor,Integrated Graphics Processor- Intel® HD Graph...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,User's manual,Realtek® ALC887 8-Channel High Definition Audi...,"128 Mb Flash ROM, UEFI AMI BIOS, PnP, SM BIOS ...",1 x PS/2 keyboard/mouse combo port(s),PRIME H310I-PLUS/CSM,Intel® Socket 1151 for 8,Intel® H310,1 x PCIe 3.0/2.0 x16 (x16 mode),Mini ITX Form Factor,Integrated Graphics Processor- Intel® HD Graph...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,User's manual,Realtek® ALC887 8-Channel High Definition Audi...,"128Mb Flash ROM, UEFI AMI BIOS, PnP, SM BIOS 3...",1 x PS/2 keyboard (purple),PRIME H310M-C/CSM,Intel® for 8,Intel® H310,1 x PCIe 3.0/2.0 x16,uATX Form Factor,Integrated Graphics Processor- Intel® HD Graph...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,User's manual,Realtek® ALC887 8-Channel High Definition Audi...,"128Mb Flash ROM, UEFI AMI BIOS, PnP, SM BIOS 3...",1 x PS/2 keyboard (purple),PRIME B360M-C/CSM,Intel® for 8,Intel® B360,1 x PCIe 3.0/2.0 x16,Micro ATX Form Factor,Integrated Graphics Processor- Intel® HD Graph...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,User's manual,Realtek® ALC887 8-Channel High Definition Audi...,"128 Mb Flash ROM, UEFI AMI BIOS, PnP, SM BIOS ...",1 x PS/2 keyboard (purple),PRIME H310M-K,Intel® Socket 1151 for 8,Intel® H310,"1 x PCIe 3.0/2.0 x16 (x16 mode, gray)",Micro ATX Form Factor,Integrated Graphics Processor- Intel® HD Graph...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,User's manual,Realtek® ALC887 8-Channel High Definition Audi...,"128 Mb Flash ROM, UEFI AMI BIOS, PnP, SM BIOS ...",1 x PS/2 keyboard (purple),PRIME H310M-E,Intel® Socket 1151 for 8,Intel® H310,"1 x PCIe 3.0/2.0 x16 (x16 mode, gray)",Micro ATX Form Factor,Integrated Graphics Processor- Intel® HD Graph...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#筆數
df.shape[0]

In [ ]:
df.head(2)

In [ ]:
df2 = df.set_index('Board name')
df2.head(2)